# 04c - Classification Models Development

**Objetivo**: Desarrollar modelos de clasificación para predecir categorías de riesgo de Alzheimer (Low, Moderate, High)
 
**Target Variable**: `risk_category`
**Clases**: Low (46.4%), Moderate (46.1%), High (7.5%)
 
**Modelos a desarrollar**:
- Logistic Regression (baseline)
- Random Forest Classifier
- Gradient Boosting (XGBoost, LightGBM)
- Support Vector Machine
- Neural Network (MLP)

---

## Importar librerías

In [1]:
import sys
import os
sys.path.append('../src/modeling')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import mlflow
import mlflow.sklearn
from sklearn.model_selection import cross_validate
from sklearn.ensemble import StackingClassifier  # Import explícito por si acaso
import warnings
warnings.filterwarnings('ignore')

import model_utils
# Importar scripts personalizados
from classification_pipeline import ClassificationPipeline
from ensemble_methods import AlzheimerEnsemble

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [10]:
print("✅ Librerías y scripts importados correctamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


✅ Librerías y scripts importados correctamente
📅 Fecha de ejecución: 2025-06-22 18:18:38


## Cargar datos

In [4]:
# Cargar datos procesados
try:
    df = pd.read_csv('../data/processed/features/alzheimer_features_selected_20250621.csv')
    print(f"📊 Dataset cargado: {df.shape}")
    
    # Verificar target variable
    if 'risk_category' in df.columns:
        print(f"🎯 Distribución de clases:")
        class_dist = df['risk_category'].value_counts()
        print(class_dist)
        print(f"📊 Porcentajes:")
        print((class_dist / len(df) * 100).round(1))
    else:
        print("❌ Error: Variable target 'risk_category' no encontrada")
        
except FileNotFoundError:
    print("❌ Error: Archivo de features no encontrado")
    print("💡 Ejecuta primero el notebook 03_feature_engineering_master.ipynb")


📊 Dataset cargado: (48466, 186)
🎯 Distribución de clases:
risk_category
Low         22501
Moderate    22345
High         3620
Name: count, dtype: int64
📊 Porcentajes:
risk_category
Low         46.4
Moderate    46.1
High         7.5
Name: count, dtype: float64


## Inicializar Pipeline de Clasificaión

In [5]:
# Inicializar pipeline de clasificación
classification_pipeline = ClassificationPipeline()

## Ejecutar Pipeline

In [6]:
# Ejecutar el pipeline completo
print("🚀 Ejecutando pipeline completo de clasificación...")
pipeline_results = classification_pipeline.run_pipeline(df, target_col='risk_category')

# Obtener resultados
results = pipeline_results['results']
cv_results = pipeline_results['cv_results']
trained_models = pipeline_results['trained_models']
best_model_name = pipeline_results['best_model']

print(f"🏆 Mejor modelo seleccionado: {best_model_name}")


🚀 Ejecutando pipeline completo de clasificación...
🚀 Iniciando pipeline de clasificación...
📊 Datos preparados: 38772 train, 9694 test
Entrenando logistic_regression...
Entrenando random_forest...
Entrenando gradient_boosting...
Entrenando svm...
   Usando subconjunto de 10000 muestras para SVM
✅ 4 modelos entrenados
📈 Evaluación completada
Validación cruzada para logistic_regression...
Validación cruzada para random_forest...
Validación cruzada para gradient_boosting...
Validación cruzada para svm...
🔄 Validación cruzada completada
🏆 Mejor modelo: gradient_boosting


2025/06/24 13:51:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


📁 Resultados registrados en MLflow
🏆 Mejor modelo seleccionado: gradient_boosting


## Recopilar métricas de rendimiento

In [7]:
# Recopilar métricas de rendimiento
performance_comparison = {}
for name, metrics in results.items():
    performance_comparison[name] = metrics['f1_weighted']

## Ensamblaje de modelos

In [ ]:
with mlflow.start_run(run_name="ensemble_classification"):
    mlflow.set_tag("model_family", "ensemble")
    mlflow.set_tag("model_type", "classification")
    
    # 0. Reconstruir datos COMPLETOS desde cero
    X_train, X_test, y_train, y_test = classification_pipeline.prepare_data(df)
    X_full = pd.concat([X_train, X_test])
    y_full = pd.concat([y_train, y_test])
    
    # 1. Definir best_models usando los modelos entrenados
    best_models = {
        'logistic_regression': trained_models['logistic_regression'],
        'random_forest': trained_models['random_forest'],
        'gradient_boosting': trained_models['gradient_boosting'],
        'svm': trained_models['svm']
    }
    
    print("🚀 Creando modelos ensemble...")
    
    # 2. Inicializar ensemble
    ensemble = AlzheimerEnsemble()
    
    # 3. Crear ensembles personalizados
    voting_clf = ensemble.create_custom_voting_ensemble(best_models)
    stacking_clf = ensemble.create_custom_stacking_ensemble(best_models)
    
    # 4. Evaluar con cross_validate (para métricas consistentes)
    from sklearn.model_selection import cross_validate
    
    print(" Evaluando Voting Classifier...")
    voting_cv = cross_validate(
        voting_clf, X_full, y_full,
        cv=3,  # Reducido para mayor velocidad
        scoring=['f1_weighted', 'accuracy'],
        n_jobs=1
    )
    
    print(" Evaluando Stacking Classifier...")
    stacking_cv = cross_validate(
        stacking_clf, X_full, y_full,
        cv=3,  # Reducido para mayor velocidad
        scoring=['f1_weighted', 'accuracy'],
        n_jobs=1
    )
    
    # 5. Entrenar modelos finales (con todos los datos)
    print(" Entrenando modelos finales...")
    voting_clf.fit(X_full, y_full)
    stacking_clf.fit(X_full, y_full)
    
    # 6. Registrar resultados
    mlflow.log_metrics({
        'voting_f1_weighted': voting_cv['test_f1_weighted'].mean(),
        'voting_accuracy': voting_cv['test_accuracy'].mean(),
        'stacking_f1_weighted': stacking_cv['test_f1_weighted'].mean(),
        'stacking_accuracy': stacking_cv['test_accuracy'].mean()
    })
    
    mlflow.sklearn.log_model(voting_clf, "voting_ensemble")
    mlflow.sklearn.log_model(stacking_clf, "stacking_ensemble")
    
    print(f""" Ensembles creados y registrados!
Voting F1: {voting_cv['test_f1_weighted'].mean():.4f}
Stacking F1: {stacking_cv['test_f1_weighted'].mean():.4f}""")

🚀 Creando modelos ensemble...
 Evaluando Voting Classifier...


## Análisis de Importancia de Características (Feature Importance)

In [ ]:
# Análisis de importancia de características (para modelos tree-based)
print("\n🔍 ANÁLISIS DE IMPORTANCIA DE FEATURES")
print("="*50)

# Obtener nombres de características
exclude_cols = ['composite_risk_score', 'risk_category']
feature_cols = [col for col in df.columns if col not in exclude_cols]

# Combinar importancias de modelos
combined_importance = pd.DataFrame()

for name, model in trained_models.items():
    if hasattr(model.named_steps['classifier'], 'feature_importances_'):
        importance = model_utils.get_feature_importance(
            model.named_steps['classifier'], feature_cols
        )
        combined_importance[name] = importance.set_index('feature')['importance']

if not combined_importance.empty:
    combined_importance['Mean_Importance'] = combined_importance.mean(axis=1)
    top_features = combined_importance.nlargest(15, 'Mean_Importance')
    
    # Visualización y guardado de resultados
    print("🎯 Top 15 Features más importantes:")
    print(top_features[['Mean_Importance']].round(4))
    
    # Visualización
    plt.figure(figsize=(10, 8))
    top_features['Mean_Importance'].plot(kind='barh', color='lightgreen')
    plt.title('Top 15 Features - Importancia Promedio')
    plt.xlabel('Importancia Promedio')
    plt.tight_layout()
    plt.show()

## Comparación Final de Modelos (agregando los ensembles)

In [ ]:
print("\n" + "="*60)
print("📊 COMPARACIÓN FINAL DE MODELOS")
print("="*60)

# Crear DataFrame comparativo
comparison_df = pd.DataFrame([
    {'Model': model, 'F1_Score_Weighted': score} 
    for model, score in performance_comparison.items()
]).sort_values('F1_Score_Weighted', ascending=False)


print(comparison_df.to_string(index=False))

# Visualización de comparación
plt.figure(figsize=(12, 6))
bars = plt.bar(comparison_df['Model'], comparison_df['F1_Score_Weighted'], 
               color='lightcoral', alpha=0.8)
plt.title('Comparación de Rendimiento - Modelos de Clasificación')
plt.xlabel('Modelo')
plt.ylabel('F1-Score Weighted')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

# Añadir valores en las barras
for bar, score in zip(bars, comparison_df['F1_Score_Weighted']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
             f'{score:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## Resumen final y Recomendaciones

In [ ]:
# Resumen final y recomendaciones
best_model = comparison_df.iloc[0]['Model']
best_score = comparison_df.iloc[0]['F1_Score_Weighted']

print("\n" + "="*60)
print("🎯 RESUMEN FINAL - MODELOS DE CLASIFICACIÓN")
print("="*60)
print(f"🏆 Mejor modelo: {best_model}")
print(f"📊 F1-Score Weighted: {best_score:.4f}")
print(f"📈 Mejora sobre baseline: {((best_score - lr_performance) / lr_performance * 100):.1f}%")
print(f"⚖️ Desbalanceamiento de clases: {imbalance_ratio:.2f}")
print(f"🎯 Total de modelos entrenados: {len(performance_comparison)}")

print("\n💡 RECOMENDACIONES:")
if best_score > 0.85:
    print("✅ Excelente rendimiento alcanzado")
elif best_score > 0.75:
    print("⚠️ Buen rendimiento, considerar optimización en Fase 5")
else:
    print("❌ Rendimiento bajo, requiere optimización en Fase 5")

if imbalance_ratio > 3:
    print("⚖️ Considerar técnicas de balanceamiento adicionales")

print("\n🔄 Listo para Fase 5: Evaluación y Optimización")
print("📁 Todos los modelos guardados en MLflow")



## Guardado de Archivos Importantes

In [ ]:
# Guardar métricas finales
final_metrics = {
    'best_model': best_model,
    'best_f1_score': best_score,
    'baseline_f1_score': lr_performance,
    'improvement_percentage': (best_score - lr_performance) / lr_performance * 100,
    'models_trained': len(performance_comparison),
    'imbalance_ratio': imbalance_ratio,
    'training_samples': len(X),
    'features_used': len(feature_cols)
}

# Guardar en archivo
pd.Series(final_metrics).to_csv('../reports/model_results/classification_summary.csv')
print("📁 Métricas guardadas en: ../reports/model_results/classification_summary.csv")

---

__Abraham Tartalos__